### Interval filteration and hrv feature generation

In [ ]:
###Inverval filteration and hrv feature generation
import hrvanalysis as hrv
import scipy.signal as signal
def bbi_filter(beat_interval):
    #remove outliers
    print("inital length")

    filtered_interval = hrv.remove_outliers(beat_interval,low_rri = 400,high_rri = 1200)
    #Replace na values with linear interpolation using forier transfer
    interpolated_interval = hrv.interpolate_nan_values(filtered_interval,interpolation_method="linear")
    # This remove ectopic beats from signal
    # nn_intervals_list = hrv.remove_ectopic_beats(interpolated_interval, method="malik")
    # # This replace ectopic beats nan values with linear interpolation
    # interpolated_nn_intervals = hrv.interpolate_nan_values(nn_intervals_list)
    nn_intervals =  hrv.get_nn_intervals(interpolated_interval)
    return nn_intervals

def hrv_generation(peak_intervals,orign_df,rr_itv,binsize=120, mechano_acoustic = True):
    """ This function takes a peak peak interval list, bin size in second,a boolean, and the original dataset with time stamps as input, then will remove the abnormal intervals and heart beats and generate the HRV features.  """
   
    if mechano_acoustic:
        ma_df = orign_df
        
        itv_1 = 0
        itv_2 = 0
        i = 0
        k = 0
        hr = pd.DataFrame(columns=['index','Hr'])
        ma_it = pd.DataFrame(columns=['index','Hr'])
        hrv_feature = pd.DataFrame()
        while i < len(peak_intervals)-1:
            
            interval = peak_intervals[i]
            # itv_1 = peaks[i+2]
            i = i+1
            itv_2 = itv_2 + rr_itv[i]
            itv_1 = interval+itv_1
            k=k+interval
            hr_df = pd.DataFrame({'index': itv_2,'Hr':interval},index = [itv_2])
            hr = pd.concat([hr,hr_df])
            ma_it = pd.concat([ma_it,hr_df])
            if k > binsize*1000:
                bined_interval = hr.Hr.values.tolist()
                feature = pd.DataFrame([hrv.get_frequency_domain_features(bined_interval)])
                feature = pd.merge_asof(feature,pd.DataFrame([hrv.get_time_domain_features(bined_interval)]),left_index=True,right_index=True)
                feature = pd. merge_asof(feature,pd.DataFrame([hrv.get_geometrical_features(bined_interval)]),left_index=True,right_index=True)
                feature = pd. merge_asof(feature,pd.DataFrame([hrv.get_csi_cvi_features(bined_interval)]),left_index=True,right_index=True)
                # try:
                #     feature = pd.DataFrame([hrv.get_frequency_domain_features(bined_interval)])
                # except:
                #     print('feature error')
                itv_1 = int(itv_1)
                feature['time'] = ma_df.index[itv_2]
                hrv_feature = pd.concat([hrv_feature,feature],ignore_index=True)
                hr = pd.DataFrame(columns=['index','Hr'])
                k=0 
            
        ma_hrv = hrv_feature.copy()
        ##return ma_hrv, ma_it
        return ma_hrv,ma_it
